In [1]:
import sys
from pathlib import Path
import os
cwd = os.getcwd()
parent = str(Path(cwd).parents[0])
sys.path.append(parent)

In [2]:
import pandas as pd
import numpy as np
import collections

In [3]:
raw_extract = pd.read_csv(parent + '/data/interventions/2022-07-27_101245-form_7-refset_34-extractions.tsv', sep='\t', encoding="ISO-8859-1")
raw_arms = pd.read_csv(parent + '/data/interventions/2022-07-27_101248-sub_arm-refset_34-sub-extraction.tsv', sep='\t', encoding="ISO-8859-1")
rec_extract = pd.read_csv(parent + '/data/interventions/2022-07-27_101251-form_7-refset_34-final.tsv', sep='\t', encoding="ISO-8859-1")
rec_arms = pd.read_csv(parent + '/data/interventions/2022-07-27_101253-sub_arm-refset_34-sub-final.tsv', sep='\t', encoding="ISO-8859-1")

In [4]:
reconciled_ids = rec_arms.db_id.to_list()

In [5]:
remove_rec = raw_arms[~raw_arms.db_id.isin(reconciled_ids)]

In [6]:
cols = ['db_id', 'trialid', 'type', 'placebo_plus_soc', 'category', 'intervention', 'intervention_plus_soc']

In [7]:
final_int = pd.concat([remove_rec[cols], rec_arms[cols]]).reset_index(drop=True)

In [8]:
final_exp = final_int[final_int.type == 'experimental'].reset_index(drop=True)

In [9]:
final_exp

,db_id,trialid,type,placebo_plus_soc,category,intervention,intervention_plus_soc
0,tri00037,ChiCTR2000029790,experimental,NaN,traditional_medicine,Traditional Medicine,undefined_soc
1,tri00030,ChiCTR2000029814,experimental,NaN,traditional_medicine,Traditional Medicine,undefined_soc
2,tri00028,ChiCTR2000029819,experimental,NaN,traditional_medicine,Traditional Medicine,undefined_soc
3,tri00022,ChiCTR2000029853,experimental,NaN,drug,Azvudine,no
4,tri00011,ChiCTR2000029941,experimental,NaN,traditional_medicine,Traditional Medicine,undefined_soc
...,...,...,...,...,...,...,...
2449,tri04829,CTRI/2020/07/026835,experimental,NaN,device,High-Flow Nasal Oxygen,no
2450,tri05118,CTRI/2020/10/028537,experimental,NaN,traditional_medicine,Traditional Medicine,undefined_soc
2451,tri05118,CTRI/2020/10/028537,experimental,NaN,traditional_medicine,Traditional Medicine,undefined_soc
2452,tri07225,ISRCTN27106947,experimental,NaN,biologics,VIR-7832,NaN


In [ ]:
#final_exp.to_csv('int_norm.csv')

In [11]:
exp = final_int[final_int.type == 'experimental']

In [13]:
ints = final_int[final_int.type == 'experimental'].intervention
int_list = ints.to_list()

In [14]:
all_ints = []
for i in int_list:
    if ';' in i:
        all_ints = all_ints + i.split(';')
    else:
        all_ints.append(i)

In [15]:
c = collections.Counter(x for x in all_ints if x)

In [16]:
int_counts = pd.DataFrame.from_dict(dict(c), orient='index').reset_index()
int_counts.columns = ['interventions', 'counts']

In [17]:
int_counts.sort_values(by='counts', ascending=False)

,interventions,counts
0,Traditional Medicine,224
30,Hydroxychloroquine,196
9,Convalescent Plasma,134
41,Ivermectin,63
21,Ritonavir,61
...,...,...
393,RD-X19 Device,1
391,Consumption of foods rich in Vitamin-D,1
390,Pinene Hydronoplactone Ribonucleic (PHR),1
389,Nifedipine,1


In [18]:
c2 = collections.Counter(x for x in int_list if x)

In [20]:
int_counts2 = pd.DataFrame.from_dict(dict(c2), orient='index').reset_index()
int_counts2.columns = ['interventions', 'counts']

In [22]:
int_counts2.sort_values(by='counts', ascending=False).to_csv(parent + '/data/interventions/int_to_norm.csv')

In [ ]:
final_exp['HCQ'] = np.where(final_exp.intervention.str.contains('Hydroxychloroquine'), 1, 0)
final_exp['IVE'] = np.where(final_exp.intervention.str.contains('Ivermectin'), 1, 0)
final_exp['CP'] = np.where(final_exp.intervention.str.contains('Convalescent Plasma'), 1, 0)

In [ ]:
final_exp[['db_id', 'HCQ', 'IVE', 'CP']].groupby('db_id').sum().to_csv(parent + '/data/interventions/common_ints.csv')

In [ ]:
ints = final_int[final_int.type == 'experimental'].intervention

In [ ]:
ints

In [ ]:
ints_exp = final_int[final_int.type == 'experimental']

In [ ]:
ints_exp[ints_exp.

In [ ]:
final_int[final_int.intervention == 'Dabur chyawanprash']

In [ ]:
int_list = ints.to_list()

In [ ]:
all_ints = []
for i in int_list:
    if ';' in i:
        all_ints = all_ints + i.split(';')
    else:
        all_ints.append(i)

In [ ]:
import collections
c = collections.Counter(x for x in all_ints if x)

In [ ]:
int_counts = pd.DataFrame.from_dict(dict(c), orient='index').reset_index()
int_counts.columns = ['interventions', 'counts']

In [ ]:
int_counts.sort_values(by='counts', ascending=False).head(10)

In [ ]:
int_counts.to_csv(parent + '/data/interventions/unique_ints.csv')